# 数据预处理

为了能用深度学习来解决现实世界的问题，我们经常从预处理原始数据开始，
在 Python 中常用的数据分析工具是 `pandas` 软件包。

## 读取数据集
举一个例子，首先创建一个人工数据集，并存储在 CSV 文件 `dataset/house_tiny.csv` 中。

In [3]:
import os

# os.makedirs(os.path.join('..', 'dataset'), exist_ok=True)
data_file = os.path.join('..', 'dataset', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

要从创建的 CSV 文件中加载原始数据集，需要导入 `pandas` 包并调用 `read_csv` 函数。
该数据集有四行三列。其中每行描述了房间数量（“NumRooms”）、巷子类型（“Alley”）和房屋价格（“Price”）。

In [4]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


## 处理缺失值

上表中的 “NaN” 项代表缺失值。
为了处理缺失的数据，典型的方法包括插值法和删除法，其中插值法用一个替代值弥补缺失值，而删除法则直接忽略缺失值。
在这里，我们将考虑插值法。

通过位置索引`iloc`，我们将`data`分成`inputs`和`outputs`，
其中前者为`data`的前两列，而后者为`data`的最后一列。
对于`inputs`中缺少的数值，我们用 0 替换“NaN”项。

In [5]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(0)
print(inputs)

   NumRooms Alley
0       0.0  Pave
1       2.0     0
2       4.0     0
3       0.0     0


对于`inputs`中的类别值或离散值，我们可以将 “NaN” 视为一个类别。
由于“巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”，`pandas`可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。
巷子类型为“Pave”的行会将“Alley_Pave”的值设置为 1，“Alley_nan”的值设置为 0。
缺少巷子类型的行会将 “Alley_Pave” 和 “Alley_nan” 分别设置为 0 和 1。

In [6]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_0  Alley_Pave  Alley_nan
0       0.0        0           1          0
1       2.0        1           0          0
2       4.0        1           0          0
3       0.0        1           0          0


## 转换为张量格式

现在`inputs`和`outputs`中的所有条目都是数值类型，它们可以转换为张量格式。

In [7]:
import torch

X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y


(tensor([[0., 0., 1., 0.],
         [2., 1., 0., 0.],
         [4., 1., 0., 0.],
         [0., 1., 0., 0.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))